# The Gravity Model
## Binary Beasts
### Shanjun Zhu, Patrick Leitloff, Su Mae Lee, Oliver Kidd

# Motivation
### Literature Review
### Gravity Model Literature
* Head, K and Mayer, T. (2014) Gravity Equations: Workhorse, Toolkit, and Cookbook
    * Defined a simple gravity model, and then expanded into other, more general models to include bilateral trade

* Anderson, J and van Wincoop, E. (2001) Gravity with Gravitas: A Solution to the Border Puzzle
    * Solution to the border problem, shows that basing empirics on theoretically grounded gravity equation affects estimates of impact of national borders on trade

* Santos Silva, J and S Tenreyro (2006), “The log of gravity ",The Review of Economics and Statistics
    * OLS estimates in log-lin relationships with heteroskedasticity cannot be interpreted as elasticities without bias, so the solution is to use a pseudo-maximum-likelihood (PML) estimation technique, that can also deal with zero values for the dependent variable



### Brexit Literature
* Dhingra, S. et al (2016) The UK Treasury analysis of 'The long-term economic impact of EU membership and the alternatives'
    * Critically examines a UK Treasury analysis of the impact of Brexit in 3 different trade agreement scenarios.
    * Argues the treasury has been too cautious in some of their assumptions, and so have understated some of the impacts of Brexit on GDP.

## **The Gravity Model of Trade:**

$T_{ij} = A\frac{Y_{i}Y_{j}}{D_{ij}}$

$T_{ij}$: Tradeflow between countries  
$Y_{m}$: GDP  
$D_{ij}$: Distance between countries

### **1. OLS Regression**

$log[T_{ij}] = \pi + log[Y_{i}] + log[Y_{j}] + log[D_{ij}] + log[EU] + log[L] + log[C] + \epsilon$

Dummy variables: EU, L, C  

EU = 1 if both exporting and importing countries are in the EU  
EU = 0 otherwise  

L = 1 for common official of primary language  
L = 0 otherwise    

C = 1 for contiguity  
C = otherwise

### **2. 2SLS Regression**
 
 **First stage regression**  
 Run an OLS regression:  
 $Y_{i} = \pi + \beta_{1}log[Y_{j}] + \beta_{2}log[COL] + \beta_{3}log[D_{ij}] + \beta_{4}log[EU] + \beta_{5}log[L] + \beta_{6}log[C] + \epsilon$  
 
 COL is a dummy variable for colonial relationship  
 COL = 1 if pair of countries were ever in colonial relationship  
 COL = 0 otherwise  
 
 for COL to be a valid instrument, it must satisfy:  
 Relevance: $\beta_{2} ≠ 0$  
 Validity: $Cov(log[COL],\epsilon) = 0$

**Second stage regression**  
Run OLS on:  
$T_{ij} = \alpha + \beta_{1}\hat{log[Y_{i}]} + {\beta_2}log[Y_{j}] + log[D_{ij}] + log log[L] + log[C] + u$  

So that we obtain the 2SLS estimator, $\hat\beta_{1}$

### **3. PPML Regression**

$T_{ij} = log[Y_{i}] + log[Y_{j}] + log[D_{ij}] + L + C$

## OLS Estimation of Model

In [3]:
import pandas as pd
import numpy as np
import gme as gme
#loading the file and inspect data
url = 'https://www.dropbox.com/s/2uha8rwc8bngcsz/servicesdataset%202.xlsx?dl=1'
df = pd.read_excel(url)

#drop null values from relevant columns in df 
trade_data = df[['exp','imp', 'trade', 'year','gdp_exp', 'gdp_imp', 'contig','comlang_off','distw','ent_cost_imp', 'ent_cost_exp', 'colony']]
trade_data = trade_data.dropna()


#include the accessibility column
trade_data['bilateral accessibility'] = np.exp(-np.log(trade_data['distw']))


#create EU dummy column
EU = ['AUT','BEL','CYP','CZE','DNK','EST','FIN','FRA','DEU','GRC','HUN','IRL','ITA','LVA','LTU','LUX','MLT','NLD','POL','SVK','SVN','ESP','SWE','GBR']
imp_EU = {}
exp_EU = {}
imp_countries = trade_data['imp'].tolist()
exp_countries = trade_data['exp'].tolist()
for country in imp_countries:
    if country in EU:
        imp_EU[country] = 1
        exp_EU[country] = 1
    else:
        imp_EU[country] = 0
        exp_EU[country] = 0
trade_data['imp_is_EU'] = trade_data['imp'].map(imp_EU)
trade_data['exp_is_EU'] = trade_data['exp'].map(exp_EU)
trade_data['between_EU'] = trade_data['imp_is_EU']*trade_data['exp_is_EU']
        
#include log GDP and log distance  
trade_data['log_gdp_exp'] = np.log(trade_data['gdp_exp'])
trade_data['log_gdp_imp'] = np.log(trade_data['gdp_imp'])
trade_data['log_distance'] = np.log(trade_data['distw'])

#create new dataframe with non-zero trade column
non_zero_trade_data = trade_data[trade_data['trade'] != 0]
non_zero_trade_data['log_trade'] = np.log(non_zero_trade_data['trade'])

<ipython-input-3-189564355c62>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_zero_trade_data['log_trade'] = np.log(non_zero_trade_data['trade'])


## Run the regression model

In [4]:
#OLS
import statsmodels.api as sm
import statsmodels.formula.api as smf
X = non_zero_trade_data[['log_gdp_exp',
               'log_gdp_imp',
               'log_distance',
               'between_EU',
               'comlang_off',
               'contig'
               ]]
    
Y = non_zero_trade_data['log_trade']
X = sm.add_constant(X) # adding a constant

model = sm.OLS(Y, X).fit()

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:              log_trade   R-squared:                       0.317
Model:                            OLS   Adj. R-squared:                  0.317
Method:                 Least Squares   F-statistic:                     813.6
Date:                Sat, 27 Feb 2021   Prob (F-statistic):               0.00
Time:                        10:27:56   Log-Likelihood:                -22482.
No. Observations:               10525   AIC:                         4.498e+04
Df Residuals:                   10518   BIC:                         4.503e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -19.8806      0.450    -44.164   

### The result shows: 
* One percent point increase in exporter GDP leads to on average 0.48 percentage point increase in trade between two countries
* One percentage point increase in distance leads to on average 0.55 percentage point decrease in trade between two countries
* Being both in the EU increases trade between two countries on average 1.55 percentage points

Confounders that may create selection bias between trade and distance, such as common language and border, are controlled in this regression. 

However, confounders that may confound trade and exporter's GDP, such as value of services transacted, are not controlled. This leads to a positive selection bias. Countries that provide more valuable services such as professional services tend to have higher GDP, and if very few countries provide such service, they also trade more. 

### This motivates us to run a 2SLS regression using whether the two countries were in colonial relationship as IV

In [5]:
from statsmodels.sandbox.regression.gmm import IV2SLS  

Instrument = non_zero_trade_data[['colony']]
end = non_zero_trade_data[['log_trade']]
exo = non_zero_trade_data[['log_gdp_exp']]

resultIV = IV2SLS(end, exo, Instrument).fit()
print(resultIV.summary())

                          IV2SLS Regression Results                           
Dep. Variable:              log_trade   R-squared:                       0.267
Model:                         IV2SLS   Adj. R-squared:                  0.267
Method:                     Two Stage   F-statistic:                       nan
                        Least Squares   Prob (F-statistic):                nan
Date:                Sat, 27 Feb 2021                                         
Time:                        10:28:49                                         
No. Observations:               10525                                         
Df Residuals:                   10524                                         
Df Model:                           1                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
log_gdp_exp     0.1495      0.005     30.492      

#### The 2SLS regression reveals that, in fact, a one percentage point increase in exporter's GDP only leads to on average 0.15 percentage point increase in trade. 

### To test the suitability of IV, it needs to satisfy three assumptions: 
* Instrument relevance, where correlation between colony and log_gdp_exp is non-zero
* colony only affects trade through log_gdp_exp
* Instrument is as good as randomly assigned. 

### Assumption 1 can be tested by regressing colony on log_gdp_exp: 

In [7]:
#colony and gdp_ex
import statsmodels.api as sm
import statsmodels.formula.api as smf
model_ce = sm.OLS(non_zero_trade_data['log_gdp_exp'], non_zero_trade_data['colony']).fit()
print(model_ce.summary())

                                 OLS Regression Results                                
Dep. Variable:            log_gdp_exp   R-squared (uncentered):                   0.049
Model:                            OLS   Adj. R-squared (uncentered):              0.049
Method:                 Least Squares   F-statistic:                              538.1
Date:                Sat, 27 Feb 2021   Prob (F-statistic):                   3.81e-116
Time:                        10:30:52   Log-Likelihood:                         -49015.
No. Observations:               10525   AIC:                                  9.803e+04
Df Residuals:                   10524   BIC:                                  9.804e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

## Compare the original dataset and dataset for OLS regression

In [8]:
print(trade_data.shape)
print(non_zero_trade_data.shape)

(25069, 19)
(10525, 20)


#### OLS or 2SLS regression forces us to use a much smaller dataset, where only countries that trade with each other are included. This leads to sample selection bias, where characteristics of countries that trade more are inflated.

For example, countries that do not trade could also be more distant, as such, if they are omitted, we will underestimate the negative effect of distance on trade

# PPML Estimation of Model

* Pseudo-Maximum-Likelihood not a common estimation method 
* requires specialist functions, which may be found in e.g. STATA

But we want Python here!

Use the GME package from the US International Trade Commission.

In [10]:
import pandas as pd
import numpy as np
import gme as gme
#loading the file and inspect data
url = 'https://www.dropbox.com/s/2uha8rwc8bngcsz/servicesdataset%202.xlsx?dl=1'
df = pd.read_excel(url)
print(df.shape)
print(df.head())


#drop null values from relevant columns in df 
trade_data = df[['exp','imp', 'trade', 'year','gdp_exp', 'gdp_imp', 'contig','comlang_off','distw','ent_cost_imp', 'ent_cost_exp', 'colony']]
trade_data = trade_data.dropna()


#include the accessibility column
trade_data['bilateral accessibility'] = np.exp(-np.log(trade_data['distw']))


#create EU dummy column
EU = ['AUT','BEL','CYP','CZE','DNK','EST','FIN','FRA','DEU','GRC','HUN','IRL','ITA','LVA','LTU','LUX','MLT','NLD','POL','SVK','SVN','ESP','SWE','GBR']
imp_EU = {}
exp_EU = {}
imp_countries = trade_data['imp'].tolist()
exp_countries = trade_data['exp'].tolist()
for country in imp_countries:
    if country in EU:
        imp_EU[country] = 1
        exp_EU[country] = 1
    else:
        imp_EU[country] = 0
        exp_EU[country] = 0
trade_data['imp_is_EU'] = trade_data['imp'].map(imp_EU)
trade_data['exp_is_EU'] = trade_data['exp'].map(exp_EU)
trade_data['between_EU'] = trade_data['imp_is_EU']*trade_data['exp_is_EU']
        
#include log GDP and log distance  
trade_data['log_gdp_exp'] = np.log(trade_data['gdp_exp'])
trade_data['log_gdp_imp'] = np.log(trade_data['gdp_imp'])
trade_data['log_distance'] = np.log(trade_data['distw'])
print(trade_data.head())

(31092, 25)
   exp  imp  year    trade sector  contig  comlang_off  comlang_ethno  colony  \
0  HUN  ABW  2005  0.00000    SER     0.0          0.0            0.0     0.0   
1  GBR  ABW  2005  0.00000    SER     0.0          0.0            1.0     0.0   
2  IRL  ABW  2005  0.00000    SER     0.0          0.0            1.0     0.0   
3  ITA  ABW  2005  4.97438    SER     0.0          0.0            0.0     0.0   
4  BEL  ABW  2005  2.48719    SER     0.0          1.0            1.0     0.0   

   comcol  ...     distw  distwces       gdp_exp  gdp_imp  etcr_exp  etcr_imp  \
0     0.0  ...  8940.499  8939.646  5.900000e+10      NaN  2.751628       NaN   
1     0.0  ...  7480.796  7479.964  1.700000e+12      NaN  0.940455       NaN   
2     0.0  ...  7123.465  7122.625  1.300000e+11      NaN  3.073227       NaN   
3     0.0  ...  8427.789  8422.343  1.100000e+12      NaN  2.036142       NaN   
4     0.0  ...  7843.255  7843.006  2.500000e+11      NaN  2.007156       NaN   

   ent_cost_im

In [11]:
#Step 1: Read the Data Frame into the Package
gme_data = gme.EstimationData(data_frame=trade_data,
                              imp_var_name='imp',
                              exp_var_name='exp',
                              trade_var_name='trade',
                              year_var_name='year')

In [12]:
#Step 2: Specify the Model
model_basic = gme.EstimationModel(estimation_data = gme_data,
                                         lhs_var = 'trade',
                                         rhs_var = ['log_gdp_exp',
                                                    'log_gdp_imp',
                                                    'log_distance',
                                                    'comlang_off',
                                                    'contig'
                                                    ])

In [13]:
#Step 3: Run the Regression and display the results
basic_estimates = model_basic.estimate()
results = basic_estimates['all']
print(results.summary())

select specification variables: ['log_gdp_exp', 'log_gdp_imp', 'log_distance', 'comlang_off', 'contig', 'trade', 'imp', 'exp', 'year'], Observations excluded by user: {'rows': 0, 'columns': 10}
drop_intratrade: no, Observations excluded by user: {'rows': 0, 'columns': 0}
drop_imp: none, Observations excluded by user: {'rows': 0, 'columns': 0}
drop_exp: none, Observations excluded by user: {'rows': 0, 'columns': 0}
keep_imp: all available, Observations excluded by user: {'rows': 0, 'columns': 0}
keep_exp: all available, Observations excluded by user: {'rows': 0, 'columns': 0}
drop_years: none, Observations excluded by user: {'rows': 0, 'columns': 0}
keep_years: all available, Observations excluded by user: {'rows': 0, 'columns': 0}
drop_missing: yes, Observations excluded by user: {'rows': 0, 'columns': 0}
Estimation began at 10:34 AM  on Feb 27, 2021
Omitted Columns: []
Estimation completed at 10:34 AM  on Feb 27, 2021
                 Generalized Linear Model Regression Results       

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1721: FutureWarning: The bic value is computed using the deviance formula. After 0.13 this will change to the log-likelihood based formula. This change has no impact on the relative rank of models compared using BIC. You can directly access the log-likelihood version using the `bic_llf` attribute. You can suppress this message by calling statsmodels.genmod.generalized_linear_model.SET_USE_BIC_LLF with True to get the LLF-based version now or False to retainthe deviance version.
  warnings.warn(


In [14]:
# Add Fixed Effects
model_fix = gme.EstimationModel(estimation_data = gme_data,
                                         lhs_var = 'trade',
                                         rhs_var = ['log_gdp_exp',
                                                    'log_gdp_imp',
                                                    'log_distance',
                                                    'comlang_off',
                                                    'contig'],
                                         fixed_effects = ['imp','exp'])
fix_estimates = model_fix.estimate()
results_fix = fix_estimates['all']
print(results_fix.summary())

select specification variables: ['log_gdp_exp', 'log_gdp_imp', 'log_distance', 'comlang_off', 'contig', 'trade', 'imp', 'exp', 'year'], Observations excluded by user: {'rows': 0, 'columns': 10}
drop_intratrade: no, Observations excluded by user: {'rows': 0, 'columns': 0}
drop_imp: none, Observations excluded by user: {'rows': 0, 'columns': 0}
drop_exp: none, Observations excluded by user: {'rows': 0, 'columns': 0}
keep_imp: all available, Observations excluded by user: {'rows': 0, 'columns': 0}
keep_exp: all available, Observations excluded by user: {'rows': 0, 'columns': 0}
drop_years: none, Observations excluded by user: {'rows': 0, 'columns': 0}
keep_years: all available, Observations excluded by user: {'rows': 0, 'columns': 0}
drop_missing: yes, Observations excluded by user: {'rows': 0, 'columns': 0}
Estimation began at 10:34 AM  on Feb 27, 2021
Omitted Columns: ['imp_fe_KIR', 'imp_fe_PLW', 'exp_fe_FSM', 'exp_fe_KIR', 'exp_fe_PLW', 'exp_fe_SLB', 'exp_fe_TON', 'imp_fe_ZWE', 'exp_fe

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1721: FutureWarning: The bic value is computed using the deviance formula. After 0.13 this will change to the log-likelihood based formula. This change has no impact on the relative rank of models compared using BIC. You can directly access the log-likelihood version using the `bic_llf` attribute. You can suppress this message by calling statsmodels.genmod.generalized_linear_model.SET_USE_BIC_LLF with True to get the LLF-based version now or False to retainthe deviance version.
  warnings.warn(


In [ ]:
'''
Generalized Linear Model Regression Results for fixed effects

==============================================================================
Dep. Variable:                  trade   No. Iterations:                     11
Model:                            GLM   Df Residuals:                    24673
Model Family:                 Poisson   Df Model:                          320
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -2.4108e+06
Covariance Type:                  HC1   Deviance:                   4.7627e+06
No. Observations:               24994   Pearson chi2:                 8.13e+06

                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
log_gdp_exp      2.2325      2.056      1.086      0.277      -1.797       6.262
log_gdp_imp     -1.9511      2.038     -0.957      0.338      -5.946       2.043
log_distance    -0.5856      0.058    -10.124      0.000      -0.699      -0.472
comlang_off      0.3306      0.192      1.723      0.085      -0.046       0.707
contig           0.2247      0.179      1.256      0.209      -0.126       0.576
'''

Great success! Log Distance coefficient estimate is reasonably close to the one expected from the literature (-0.56)

# Obstacles
## Github
* As someone who had not used Github before, at first it was slightly confusing
* However, after using it for this project, I can now see why it is very useful in group projects and using it throughout the project has allowed us to get more comfortable with it

## Tips for Our Successors

* Don't underestimate the importance of a solid lit review!

* Expect error messages, setbacks and results that don't make sense

* Python is a great tool, but not a specialist statistical software! Consider running PPML models in STATA

* Have fun and enjoy the process!